# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_file = "Output/city_output.csv"
city_df = pd.read_csv(city_file)
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,saskylakh,71.92,114.08,37.78,94,100,18.39,RU,1595801012
1,yellowknife,62.46,-114.35,73.99,43,20,10.29,CA,1595801013
2,kaseda,31.42,130.32,75.20,88,75,8.90,JP,1595801013
3,mar del plata,-38.00,-57.56,52.00,71,100,12.75,AR,1595800751
4,azul,-36.78,-59.86,47.52,69,98,6.87,AR,1595801014


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = city_df[["Lat", "Lng"]].astype(float)

# Store humidity data
humidity = city_df["Humidity"].astype(float)

In [4]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create a humidity Heatmap layer
# fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 100)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_cities = city_df[city_df["Max Temp"] < 85] 
perfect_cities = perfect_cities[perfect_cities["Max Temp"] > 75]
perfect_cities = perfect_cities[perfect_cities["Wind Speed"] < 15]
perfect_cities = perfect_cities[perfect_cities["Wind Speed"] > 5]
perfect_cities = perfect_cities[perfect_cities["Cloudiness"] == 0]
hotel_df = perfect_cities.dropna()
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
291,invermere,50.52,-116.04,79.00,31,0,5.01,CA,1595801087
335,yinchuan,38.47,106.27,77.40,31,0,9.78,CN,1595801097
398,williston,48.15,-103.62,80.01,36,0,8.99,US,1595801082
400,loralai,30.37,68.60,79.88,53,0,8.59,PK,1595801112
406,bardiyah,31.76,25.09,75.49,88,0,9.91,LY,1595801114
496,nalut,30.33,10.85,83.21,26,0,14.27,LY,1595801135
512,manavgat,36.79,31.44,80.60,83,0,5.12,TR,1595801139
534,marsa matruh,31.35,27.25,77.00,83,0,5.82,EG,1595801145


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "None"
        pass

In [7]:
# hotel_df = hotel_df.drop(hotel_df[hotel_df["Hotel Name"] == None].index, inplace = True)
hotel_df = hotel_df[hotel_df["Hotel Name"] != "None"]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
291,invermere,50.52,-116.04,79.00,31,0,5.01,CA,1595801087,The Kanata Inns - Invermere Hotel
335,yinchuan,38.47,106.27,77.40,31,0,9.78,CN,1595801097,Kempinski Hotel Yinchuan
398,williston,48.15,-103.62,80.01,36,0,8.99,US,1595801082,Best Western Plus Williston Hotel & Suites
400,loralai,30.37,68.60,79.88,53,0,8.59,PK,1595801112,Saddam Hotel & Resturant
406,bardiyah,31.76,25.09,75.49,88,0,9.91,LY,1595801114,Hotel Burdy
512,manavgat,36.79,31.44,80.60,83,0,5.12,TR,1595801139,Bosphorus Sorgun Hotel
534,marsa matruh,31.35,27.25,77.00,83,0,5.82,EG,1595801145,Beau Site


In [8]:
# Plot markers on top of the heatmap. 
# Get coordinates from data frame
coordinates = []
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    coordinate = (lat, lng)
    coordinates.append(coordinate)

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='1px', wi…